<h1> Andrey Lukyanenko NER talk (sberloga) adption for ru WikiNer dataset

In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from sklearn.model_selection import train_test_split
import eli5

import pandas as pd

from collections import Counter
from ast import literal_eval


In [2]:
DATA_PATH = '../datasets'

<h2>Loading dataset

This dataset is taken from Annotated Corpus for Named Entity Recognition by Abhinav Walia dataset and then processed.

Annotated Corpus for Named Entity Recognition is annotated Corpus for Named Entity Recognition using GMB(Groningen Meaning Bank) corpus for entity classification with enhanced and popular features by Natural Language Processing applied to the data set.

Essential info about entities:

geo = Geographical Entity
org = Organization
per = Person
gpe = Geopolitical Entity
tim = Time indicator
art = Artifact
eve = Event
nat = Natural Phenomenon

In [3]:
df = pd.read_csv(f'{DATA_PATH}/ner.csv')

In [4]:
df.POS = df.POS.apply(literal_eval)
df.Tag = df.Tag.apply(literal_eval)

In [5]:
df.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"[NNS, IN, NNS, VBN, IN, DT, NN, VBD, DT, NNS, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"[PRP, VBD, IN, DT, NNS, IN, NN, TO, DT, NN, IN...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","[NNS, VBD, DT, NN, IN, NNS, IN, CD, IN, NNS, V...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,Sentence: 5,The protest comes on the eve of the annual con...,"[DT, NN, VBZ, IN, DT, NN, IN, DT, JJ, NN, IN, ...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,..."


In [6]:
df.shape

(47959, 4)

In [7]:
df.Sentence[11]

'The European Union , with U.S. backing , has threatened to refer Iran to the U.N. Security Council , which could impose sanctions if it finds Tehran has violated the Nuclear Non-Proliferation treaty .'

<h2>Sklearn_crfsuite

In [8]:
#transform data to tuples like (word, POS, ent)

data = []
for _, row in df.iterrows():
    tokens = [token for token in zip(row['Sentence'].split(), row['POS'], row['Tag'])]
    data.append(tokens)

In [9]:
data[:10]

[[('Thousands', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('demonstrators', 'NNS', 'O'),
  ('have', 'VBP', 'O'),
  ('marched', 'VBN', 'O'),
  ('through', 'IN', 'O'),
  ('London', 'NNP', 'B-geo'),
  ('to', 'TO', 'O'),
  ('protest', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('war', 'NN', 'O'),
  ('in', 'IN', 'O'),
  ('Iraq', 'NNP', 'B-geo'),
  ('and', 'CC', 'O'),
  ('demand', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('withdrawal', 'NN', 'O'),
  ('of', 'IN', 'O'),
  ('British', 'JJ', 'B-gpe'),
  ('troops', 'NNS', 'O'),
  ('from', 'IN', 'O'),
  ('that', 'DT', 'O'),
  ('country', 'NN', 'O'),
  ('.', '.', 'O')],
 [('Families', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('soldiers', 'NNS', 'O'),
  ('killed', 'VBN', 'O'),
  ('in', 'IN', 'O'),
  ('the', 'DT', 'O'),
  ('conflict', 'NN', 'O'),
  ('joined', 'VBD', 'O'),
  ('the', 'DT', 'O'),
  ('protesters', 'NNS', 'O'),
  ('who', 'WP', 'O'),
  ('carried', 'VBD', 'O'),
  ('banners', 'NNS', 'O'),
  ('with', 'IN', 'O'),
  ('such', 'JJ', 'O'),
  ('slogans', 'NNS', 'O'),
  (

In [10]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,            
        })
    else:
        features['BOS'] = True
    
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]   

In [11]:
train_data, test_data = train_test_split(data, test_size=0.33, random_state=42)

In [12]:
len(train_data), len(test_data)

(32132, 15827)

In [15]:
X_train = [sent2features(s) for s in train_data]
y_train = [sent2labels(s) for s in train_data]
X_test = [sent2features(s) for s in test_data]
y_test = [sent2labels(s) for s in test_data]

In [16]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True,
)

crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|███████████████████████████████████████| 32132/32132 [00:04<00:00, 6701.17it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 122704
Seconds required: 1.224

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=1.68  loss=1199945.08 active=121818 feature_norm=1.00
Iter 2   time=2.51  loss=708691.89 active=121119 feature_norm=3.68
Iter 3   time=0.83  loss=584528.57 active=115569 feature_norm=3.24
Iter 4   time=2.53  loss=444237.97 active=117394 feature_norm=2.84
Iter 5   time=0.85  loss=386270.45 active=119803 feature_norm=3.39
Iter 6   time=0.85  loss=311683.50 active=120685 feature_norm=4.64
Iter 7   time=0.84  loss=237150.42 active=110405 feature_norm=6.83
Iter 8   time=0.83  loss=195103.04 active=98985 feature_norm=9.32
Iter 9   time=0.84  loss=168813.70 active=97119 feature_norm=1

C:\Users\Asus\anaconda3\envs\nlp_env\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100, verbose=True)

In [17]:
all_ents = list(crf.classes_)
ents = [ent for ent in all_ents if ent != 'O']

In [18]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_ents)

0.9713537781004932

In [19]:
# y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=ents)

0.8524923786818313

In [20]:
# group B and I results
sorted_ents = sorted(
    ents,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_ents, digits=3, zero_division=0
))

C:\Users\Asus\anaconda3\envs\nlp_env\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['B-art', 'I-art', 'B-eve', 'I-eve', 'B-geo', 'I-geo', 'B-gpe', 'I-gpe', 'B-nat', 'I-nat', 'B-org', 'I-org', 'B-per', 'I-per', 'B-tim', 'I-tim'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

       B-art      0.388     0.142     0.208       134
       I-art      0.071     0.021     0.033        94
       B-eve      0.460     0.296     0.360        98
       I-eve      0.345     0.216     0.266        88
       B-geo      0.865     0.906     0.885     12623
       I-geo      0.828     0.802     0.815      2461
       B-gpe      0.970     0.942     0.956      5271
       I-gpe      0.830     0.565     0.672        69
       B-nat      0.737     0.346     0.471        81
       I-nat      0.692     0.429     0.529        21
       B-org      0.793     0.742     0.767      6473
       I-org      0.823     0.803     0.813      5482
       B-per      0.846     0.823     0.834      5651
       I-per      0.850     0.902     0.875      5720
       B-tim      0.929     0.877     0.902      6756
       I-tim      0.827     0.760     0.792      2248

   micro avg      0.861     0.848     0.855     53270
   macro avg      0.703   

In [21]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-nat  -> I-nat   6.035688
B-art  -> I-art   5.784189
I-art  -> I-art   5.570932
B-eve  -> I-eve   5.528957
I-eve  -> I-eve   5.147286
B-geo  -> I-geo   5.027504
B-gpe  -> I-gpe   4.866447
B-tim  -> I-tim   4.745348
I-tim  -> I-tim   4.647870
I-nat  -> I-nat   4.338504
I-gpe  -> I-gpe   4.213156
O      -> O       4.118929
I-geo  -> I-geo   3.997150
I-org  -> I-org   3.992198
B-per  -> I-per   3.803928
B-org  -> I-org   3.588613
I-per  -> I-per   2.846286
O      -> B-geo   1.590190
O      -> B-per   1.502896
O      -> B-tim   1.463006

Top unlikely transitions:
I-org  -> B-org   -4.075165
O      -> I-art   -4.117126
B-tim  -> I-org   -4.218720
B-org  -> B-org   -4.330540
B-gpe  -> I-geo   -4.395669
I-per  -> I-org   -4.448331
B-per  -> I-org   -4.519423
B-tim  -> B-tim   -4.643978
B-geo  -> I-per   -4.664962
I-org  -> I-per   -4.800842
B-geo  -> I-org   -5.023734
B-gpe  -> I-org   -5.288739
B-org  -> I-per   -5.651856
B-gpe  -> B-gpe   -5.666952
O      -> I-per  

In [22]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
7.998412 B-tim    word[-3:]:Day
7.796033 O        word.lower():last
7.647285 B-per    word.lower():president
7.201348 O        word.lower():month
7.161645 B-tim    word[-3:]:day
6.973766 B-org    word.lower():philippine
6.899734 B-tim    word[-3:]:70s
6.556115 B-geo    word.lower():caribbean
6.516715 B-per    word.lower():vice
6.509271 B-gpe    word.lower():niger
6.402200 B-tim    word[-3:]:9th
6.227336 B-gpe    word.lower():afghan
6.223352 B-gpe    word.lower():nepal
6.204919 O        BOS
5.895395 B-nat    word.lower():katrina
5.715317 B-tim    word.lower():one-fifth
5.660981 B-org    word.lower():hamas
5.610139 B-tim    word.lower():multi-candidate
5.606653 B-tim    word.lower():weekend
5.507496 B-org    -1:word.lower():rice
5.439438 B-tim    word.lower():august
5.368615 B-per    word.lower():prime
5.363698 B-tim    word.lower():titan
5.339650 B-tim    word.lower():one-year
5.332534 B-geo    word.lower():europe
5.310539 B-tim    word.lower():february
5.283398 B-tim    w

In [23]:
eli5.show_weights(crf, top=10)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,4.119,0.367,-4.117,0.604,-3.931,1.59,-7.269,0.108,-2.981,0.282,-2.96,1.171,-7.161,1.503,-5.934,1.463,-6.302
B-art,-0.312,0.0,5.784,0.0,-0.106,-0.132,-1.163,-1.196,-0.073,0.0,0.0,0.0,-2.32,-1.448,-1.939,-0.733,-0.911
I-art,-0.856,-0.828,5.571,-0.006,-0.033,-0.28,-1.395,-1.109,0.0,0.0,0.0,-1.263,-2.146,-1.65,-2.209,-0.798,-1.327
B-eve,-0.543,0.0,-0.304,-0.246,5.529,-0.968,-1.026,-1.462,-0.073,-0.223,-0.094,-1.78,-2.051,-2.32,-1.895,0.077,-0.986
I-eve,-0.227,0.0,-0.055,-1.84,5.147,-1.056,-1.079,-0.693,0.0,0.0,0.0,-0.875,-1.452,-1.579,-1.506,-1.391,-1.385
B-geo,0.657,0.383,-2.301,0.0,-2.189,-3.685,5.028,-0.117,-2.852,-0.241,-1.287,-0.549,-5.024,-1.667,-4.665,1.043,-3.208
I-geo,0.325,1.134,-1.337,-0.549,-1.223,-2.593,3.997,-1.196,-1.742,0.0,-0.518,-0.754,-3.743,-1.065,-3.381,0.283,-2.235
B-gpe,0.783,-1.07,-1.989,-1.032,-2.632,0.172,-4.396,-5.667,4.866,-0.287,-1.185,0.846,-5.289,-0.734,-4.057,-0.523,-2.742
I-gpe,-0.053,0.0,0.0,0.0,0.0,-1.245,-0.684,-0.65,4.213,0.0,0.0,-0.98,-1.508,-0.461,-0.848,-1.26,-0.643
B-nat,-0.52,0.0,0.0,0.0,0.0,0.781,-0.193,-0.644,0.0,-0.157,6.036,-0.453,-0.84,-1.532,-1.372,-0.912,-0.26


In [25]:
#change regularization 

%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=10,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True,
)

crf.fit(X_train, y_train)

UsageError: Line magic function `%%time` not found.


In [ ]:
eli5.show_weights(crf, top=10)

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_ents), metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=ents)

<h2>Spacy

In [ ]:
with open(f'{DATA_PATH}/ner.bio', 'w', encoding='utf8') as file:
    for _, row in df.iterrows():
        for word, _, ent in zip(row['Sentence'].split(), row['POS'], row['Tag']):
            file.write(f'{word}\t{ent}\n')
        file.write('\n')

In [ ]:
with open(f'{DATA_PATH}/ner_train.bio', 'w', encoding='utf8') as file:
    for sent in train_data:
        for token in sent:
            file.write(f'{token[0]}\t{token[2]}\n')
        file.write('\n')

In [ ]:
with open(f'{DATA_PATH}/ner_test.bio', 'w', encoding='utf8') as file:
    for sent in test_data:
        for token in sent:
            file.write(f'{token[0]}\t{token[2]}\n')
        file.write('\n')

In [ ]:
!python -m spacy init config base_config.cfg -p ner

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy convert ../ner.bio ../spacy_data -t spacy -c ner

In [ ]:
!python -m spacy convert ../ner_train.bio ../spacy_data -t spacy -c ner

In [ ]:
!python -m spacy convert ../ner_test.bio ../spacy_data -t spacy -c ner

In [ ]:
# works in cli
# !python -m spacy train config.cfg --output ./output --paths.train ../spacy_data/ner_train.spacy --paths.dev ../spacy_data/ner_test.spacy

In [28]:
!python -m spacy evaluate output/model-best ../spacy_data/ner_test.spacy

[i] Using CPU

================================== Results ==================================

TOK     -    
NER P   84.55
NER R   83.37
NER F   83.96
SPEED   4614 


=============================== NER (per type) ===============================

          P       R       F
tim   88.48   86.44   87.45
geo   86.12   89.26   87.66
org   74.61   69.87   72.17
per   78.06   75.24   76.63
gpe   95.38   94.35   94.86
art   25.33   14.18   18.18
eve   47.37   27.55   34.84
nat   60.71   20.99   31.19



2022-05-02 19:12:38.309594: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
